Build Association Rules

In [5]:
import pandas as pd

In [6]:
df = pd.read_excel('Online Retail.xlsx')

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Because our goal is to recommend products purchased together by examining the frequency by which different items are purchased together we only need information that identifies individual orders and individual products.  Because it will be more convenient for display purposes we will also use the Description.  We don't need the rest of the columns for this project.

We will keep two DataFrames for this.

One for Building the recommendation system with the following features:
- `InvoiceNo`
- `StockCode`

And one for matching the description to the `StockCode`:
- `StockCode`
- `Description`



In [89]:
# Modify StockCode to always be a string

# Prepend '_' to StockCode
df['StockCode'] = df['StockCode'].apply(lambda x: '_'+str(x))

In [91]:
# DataFrame for building the recommendation system
orders = df[['InvoiceNo', 'StockCode']]
orders.head()

,InvoiceNo,StockCode
0,536365,_85123A
1,536365,_71053
2,536365,_84406B
3,536365,_84029G
4,536365,_84029E


In [211]:
# DataFrame for retrieving product descriptions
products = df[['StockCode', 'Description']].copy()
products.head()

,StockCode,Description
0,_85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,_71053,WHITE METAL LANTERN
2,_84406B,CREAM CUPID HEARTS COAT HANGER
3,_84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,_84029E,RED WOOLLY HOTTIE WHITE HEART.


In [212]:
products['StockCode'] = products['StockCode'].str.upper()

In [214]:
products = products[~products.duplicated()]
products.head()

,StockCode,Description
0,_85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,_71053,WHITE METAL LANTERN
2,_84406B,CREAM CUPID HEARTS COAT HANGER
3,_84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,_84029E,RED WOOLLY HOTTIE WHITE HEART.


In [225]:
products[products['StockCode'] == '_23236']

,StockCode,Description
218408,_23236,DOILEY STORAGE TIN
220496,_23236,DOILEY BISCUIT TIN
290770,_23236,STORAGE TIN VINTAGE DOILEY
292790,_23236,STORAGE TIN VINTAGE DOILY


In [218]:
# Drop descriptions that are not uppercase
products = products[
    products['Description'].str.upper() == products['Description']
]

In [228]:
# Keep only the first Description of each product
products = products[~products.duplicated(subset=['StockCode'])]
products

,StockCode,Description
0,_85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,_71053,WHITE METAL LANTERN
2,_84406B,CREAM CUPID HEARTS COAT HANGER
3,_84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,_84029E,RED WOOLLY HOTTIE WHITE HEART.
...,...,...
504104,_23561,SET OF 6 RIBBONS PARTY
507867,_23609,SET 10 CARDS SNOWY ROBIN 17099
512588,_23617,SET 10 CARDS SWIRLY XMAS TREE 17104
527065,_90214U,"LETTER ""U"" BLING KEY RING"


In [223]:
products[products['StockCode'] == '_21755']

,StockCode,Description
18,_21755,LOVE BUILDING BLOCK WORD


In [187]:
# Number of unique products
len(products)

4097

# Number of orders

In [192]:
orders

,InvoiceNo,StockCode
0,536365,"[_85123A, _71053, _84406B, _84029G, _84029E, _..."
1,536366,"[_22633, _22632]"
2,536367,"[_84879, _22745, _22748, _22749, _22310, _8496..."
3,536368,"[_22960, _22913, _22912, _22914]"
4,536369,[_21756]
...,...,...
25895,C581484,[_23843]
25896,C581490,"[_22178, _23144]"
25897,C581499,[_M]
25898,C581568,[_21258]


In [188]:
# total number of orders
orders['InvoiceNo'].nunique()

25900

In [190]:
# orders with more than one item
num_items_in_order = orders.groupby('InvoiceNo').count()
num_items_in_order.columns = ['Count']
num_items_in_order

,Count
InvoiceNo,
536365,1
536366,1
536367,1
536368,1
536369,1
...,...
C581484,1
C581490,1
C581499,1


In [191]:
len(num_items_in_order[num_items_in_order['Count'] > 1])

0

There are 20k orders with more than one product.  That is about 80% of all orders.  People in this store often buy items together.  We are going to help new customers out by showing them which products are commonly purchased together.

# Restructure the data
We would like each Invoice Number to give us a list of stock codes.

In [102]:
def string_list(x):
    return [str(i) for i in x]

orders = orders.groupby('InvoiceNo')['StockCode'].apply(list).reset_index()
orders.head()

,InvoiceNo,StockCode
0,536365,"[_85123A, _71053, _84406B, _84029G, _84029E, _..."
1,536366,"[_22633, _22632]"
2,536367,"[_84879, _22745, _22748, _22749, _22310, _8496..."
3,536368,"[_22960, _22913, _22912, _22914]"
4,536369,[_21756]


In [27]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25900 entries, 0 to 25899
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   InvoiceNo  25900 non-null  object
 1   StockCode  25900 non-null  object
dtypes: object(2)
memory usage: 404.8+ KB


In [49]:
for item in orders['StockCode'][0]:
    print(type(item))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [103]:
from mlxtend.preprocessing import TransactionEncoder

In [104]:
te = TransactionEncoder()

te_ary = te.fit(orders['StockCode']).transform(orders['StockCode'])
df_ary = pd.DataFrame(te_ary, columns =te.columns_)
df_ary.head()

,_10002,_10080,_10120,_10123C,_10123G,_10124A,_10124G,_10125,_10133,_10134,...,_M,_PADS,_POST,_S,_gift_0001_10,_gift_0001_20,_gift_0001_30,_gift_0001_40,_gift_0001_50,_m
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
from mlxtend.frequent_patterns import apriori

In [196]:
%%timeit -n1 -r1

frequent_itemsets_apriori = apriori(df_ary, min_support=0.01, use_colnames=True)

2min 52s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [239]:
frequent_itemsets_apriori = apriori(df_ary, min_support=0.01, use_colnames=True)

In [241]:
frequent_itemsets_apriori.sort_values( ['itemsets'])

,support,itemsets
0,0.020193,(_15036)
715,0.019575,"(_20728, _22384)"
716,0.010695,"(_20728, _22662)"
717,0.014208,"(_23206, _20728)"
718,0.011042,"(_23207, _20728)"
...,...,...
368,0.011042,(_22840)
369,0.016371,(_22844)
370,0.012548,(_22845)
356,0.022239,(_22776)


In [ ]:
from mlxtend.frequent_patterns import fpgrowth

In [229]:
%%timeit -n1 -r1

frequent_itemsets = fpgrowth(df_ary, min_support=0.01, use_colnames=True)#, max_len=2)

5.43 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
frequent_itemsets_fpgrowth = fpgrowth(df_ary, min_support=0.01, use_colnames=True)#, max_len=2)

In [237]:
frequent_itemsets_fpgrowth.sort_values(['itemsets'])

,support,itemsets
0,0.086718,(_85123A)
714,0.013822,"(_20725, _22662)"
715,0.011004,"(_22662, _20726)"
716,0.010270,"(_22662, _22384)"
717,0.013089,"(_22383, _22662)"
...,...,...
359,0.013861,(_22348)
358,0.014170,(_21990)
357,0.014286,(_20975)
363,0.013205,(_84536A)


In [ ]:
# try sorting the sets to see if they are the same.

In [128]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(_21755),(_21754),0.024363,0.030386,0.011467,0.470681,15.490025,0.010727,1.831815
1,(_21754),(_21755),0.030386,0.024363,0.011467,0.377382,15.490025,0.010727,1.566993
2,(_22745),(_22748),0.016448,0.016988,0.012124,0.737089,43.387751,0.011844,3.738955
3,(_22748),(_22745),0.016988,0.016448,0.012124,0.713636,43.387751,0.011844,3.434626
4,(_22960),(_22720),0.047104,0.056448,0.014865,0.315574,5.590534,0.012206,1.378603
...,...,...,...,...,...,...,...,...,...
831,(_23293),(_23295),0.021506,0.017336,0.011853,0.551167,31.793373,0.011480,2.189376
832,(_23296),(_23293),0.014826,0.021506,0.010077,0.679688,31.604859,0.009758,3.054811
833,(_23293),(_23296),0.021506,0.014826,0.010077,0.468582,31.604859,0.009758,1.853857
834,(_23355),(_22112),0.030502,0.033436,0.010463,0.343038,10.259450,0.009443,1.471263


In [129]:
ar = association_rules(frequent_itemsets, metric="lift", min_threshold=2)
ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(_84879),(_85123A),0.056680,0.086718,0.012510,0.220708,2.545124,0.007595,1.171939
1,(_85123A),(_84879),0.086718,0.056680,0.012510,0.144256,2.545124,0.007595,1.102340
2,(_84879),(_22423),0.056680,0.083861,0.010425,0.183924,2.193197,0.005672,1.122614
3,(_22423),(_84879),0.083861,0.056680,0.010425,0.124309,2.193197,0.005672,1.077230
4,(_21755),(_21754),0.024363,0.030386,0.011467,0.470681,15.490025,0.010727,1.831815
...,...,...,...,...,...,...,...,...,...
1333,(_23293),(_23295),0.021506,0.017336,0.011853,0.551167,31.793373,0.011480,2.189376
1334,(_23296),(_23293),0.014826,0.021506,0.010077,0.679688,31.604859,0.009758,3.054811
1335,(_23293),(_23296),0.021506,0.014826,0.010077,0.468582,31.604859,0.009758,1.853857
1336,(_23355),(_22112),0.030502,0.033436,0.010463,0.343038,10.259450,0.009443,1.471263


In [165]:
# recomendations for (_84879)
recs = ar[ar['antecedents'] == {'_23293'}].consequents.apply(lambda x: next(iter(x)))
recs

1333    _23295
1335    _23296
Name: consequents, dtype: object

In [168]:
#products = products.set_index('StockCode')

In [173]:
print('Antecedent:', products.loc['_23293'][0])

print()
print('Recommendations:')
for _, rec in recs[:6].iteritems():
    print(products.loc[rec][0])

Antecedent: SET OF 12 FAIRY CAKE BAKING CASES

Recommendations:
SET OF 12 MINI LOAF BAKING CASES
SET OF 6 TEA TIME BAKING CASES


# Confidence

Confidence is a measure of how confident we are in our prediction.  It defined by:

$$ \frac{\text{Number of transactions with both Item A AND Item B}}{\text{Number of transactions with Item A}}$$

This accounts for situations where some items are often purchased.  For example if everyone buys batteries with their order, we can't really recommend any of those items when someone buys batteries because the recommendations aren't useful.

In [ ]:
def calculate_confidence(itema, itemb, df):
    df